## Goals of this project:
1. How much impact does being late or too spaced out at the first stop have downstream?
2. What is the impact of the layover at the start of the trip (the difference between the first top arrival and departure time)? Does more layover lead to more stable headways (lower values for % headway deviation)?
3. How closely does lateness (ADHERENCE) correlate to headway?
4. What is the relationship between distance or time travelled since the start of a given trip and the headway deviation? Does headway become less stable the further along the route the bus has travelled?
5. How much of a factor does the driver have on headway and on-time performance? The driver is indicated by the OPERATOR variable.
6. How does direction of travel, route, or location affect the headway and on-time performance?
7. How does time of day or day of week affect headway and on-time performance? Can you detect an impact of school schedule on headway deviation (for certain routes and at certain times of day)?
8. Does weather have any effect on headway or on-time performance? To help answer this question, the file bna_2022.csv contains historical weather data recorded at Nashville International Airport.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_theme(style='whitegrid', palette=('Paired'), font='sans-serif', font_scale=1, color_codes=True, rc=None)

In [ ]:
headway = pd.read_csv('../data/Headway Data.csv')
weather = pd.read_csv('../data/bna_weather.csv')

In [ ]:
headway_df = headway[['ADHERENCE_ID', 'DATE', 'ROUTE_ABBR', 'BLOCK_ABBR', 'OPERATOR', 'TRIP_ID', 'ROUTE_DIRECTION_NAME', 'TIME_POINT_ABBR', 'ROUTE_STOP_SEQUENCE', 'LATITUDE', 'LONGITUDE', 'SCHEDULED_TIME', 'ACTUAL_ARRIVAL_TIME', 'ACTUAL_DEPARTURE_TIME', 'ADHERENCE', 'SCHEDULED_HDWY', 'ACTUAL_HDWY', 'HDWY_DEV']]
weather_df = weather[['Date', 'temp', 'wx_phrase']]

In [ ]:
headway_df.columns = ['adh_id', 'date', 'rte_abbr', 'blk_abbr', 'opr', 'trip_id', 'rte_dir_name', 'time_pt_abbr', 'rte_stop_seq', 'lat', 'log', 'schd_time', 'act_arrvl_time', 'act_depart', 'adh', 'schd_hdwy', 'act_hdwy', 'hdwy_dev']
display(headway_df)

In [ ]:
weather_df.columns = ['date', 'temp', 'weather']
display(weather_df)

In [ ]:
print(weather_df['weather'].unique())

## Q1. How much impact does being late or too spaced out at the first stop have downstream?

In [ ]:
q1df = headway_df[['date', 'trip_id', 'schd_time', 'act_arrvl_time', 'act_depart', 'adh']]
display(q1df)

#### all the first trips and their time

In [ ]:
first_trip = (q1df.melt(id_vars=['trip_id', 'date'], value_vars=['schd_time'])
         .groupby(['trip_id', 'date'])['value']
         .agg(['min']))
display(first_trip)

#### first trips adh

In [ ]:
first_trip_adh = pd.merge(first_trip, q1df[['date', 'trip_id', 'schd_time', 'adh']], left_on=['date', 'trip_id', 'min'], right_on=['date', 'trip_id', 'schd_time'], how='inner')
display(first_trip_adh)

#### created column that shows wether bus is late, early, or on time

In [ ]:
def categorise(row):  
    if row['adh'] > 0:
        return 'early'
    elif row['adh'] < 0:
        return 'late'
    elif row['adh'] == 0:
        return 'on time'
    return 'null'

In [ ]:
first_trip_adh['on_time'] = first_trip_adh.apply(lambda row: categorise(row), axis=1)
display(first_trip_adh)

#### anti-join merge, the start of getting everything but the first stop

In [ ]:
not_first_trip = (pd.merge(first_trip_adh, q1df[['date', 'trip_id', 'schd_time', 'adh']], left_on=['date', 'trip_id', 'min'], right_on=['date', 'trip_id', 'schd_time'], how='outer', indicator=True)
     .query('_merge != "both"')
     .drop('_merge', 1))
display(not_first_trip)

#### the adh for every stop expect the first one

In [ ]:
not_first_trip.drop(columns=['min', 'schd_time_x', 'adh_x', 'on_time', 'schd_time_y'], inplace=True)
not_first_trip.rename(columns={'adh_y':'nft_adh'}, inplace=True)
display(not_first_trip)

#### the avg adh for every stop excluding the first stop

In [ ]:
not_first_trip_avg_adh = not_first_trip.groupby(['date', 'trip_id'], as_index=False)['nft_adh'].mean()
not_first_trip_avg_adh.rename(columns={'nft_adh':'nft_avg_adh'}, inplace=True)
display(not_first_trip_avg_adh)

#### merge of the first stops df and not the first stops df. 

In [ ]:
trip_id_adhs = pd.merge(first_trip_adh, not_first_trip_avg_adh, on=['date','trip_id'], how='inner')
trip_id_adhs.drop(columns=['min', 'schd_time'], inplace=True)
display(trip_id_adhs)

## Q1, but with route numbers instead of trip id

In [ ]:
q1df2 = headway_df[['date', 'rte_abbr', 'schd_time', 'act_arrvl_time', 'act_depart', 'adh']]
display(q1df2)

In [ ]:
first_trip2 = (q1df2.melt(id_vars=['rte_abbr', 'date'], value_vars=['schd_time'])
         .groupby(['rte_abbr', 'date'])['value']
         .agg(['min']))
display(first_trip2)

In [ ]:
first_trip_adh2 = pd.merge(first_trip2, q1df2[['date', 'rte_abbr', 'schd_time', 'adh']], left_on=['date', 'rte_abbr', 'min'], right_on=['date', 'rte_abbr', 'schd_time'], how='inner')
display(first_trip_adh2)

In [ ]:
first_trip_adh2['on_time'] = first_trip_adh2.apply(lambda row: categorise(row), axis=1)
display(first_trip_adh2)

In [ ]:
not_first_trip2 = (pd.merge(first_trip_adh2, q1df2[['date', 'rte_abbr', 'schd_time', 'adh']], left_on=['date', 'rte_abbr', 'min'], right_on=['date', 'rte_abbr', 'schd_time'], how='outer', indicator=True)
     .query('_merge != "both"')
     .drop('_merge', 1))
display(not_first_trip2)

In [ ]:
not_first_trip2.drop(columns=['min', 'schd_time_x', 'adh_x', 'on_time', 'schd_time_y'], inplace=True)
not_first_trip2.rename(columns={'adh_y':'nft_adh'}, inplace=True)
display(not_first_trip2)

In [ ]:
not_first_trip_avg_adh2 = not_first_trip2.groupby(['date', 'rte_abbr'], as_index=False)['nft_adh'].mean()
not_first_trip_avg_adh2.rename(columns={'nft_adh':'nft_avg_adh'}, inplace=True)
display(not_first_trip_avg_adh2)

In [ ]:
rte_adhs = pd.merge(first_trip_adh2, not_first_trip_avg_adh2, on=['date','rte_abbr'], how='inner')
rte_adhs.drop(columns=['min', 'schd_time'], inplace=True)
display(rte_adhs)

#### making table with no nulls

In [ ]:
rte_adhs.dropna(how='any',axis=0, inplace=True) 
display(rte_adhs)

#### new column

In [ ]:
def categorise(row):  
    if row['nft_avg_adh'] > 0:
        return 'early'
    elif row['nft_avg_adh'] < 0:
        return 'late'
    elif row['nft_avg_adh'] == 0:
        return 'on time'
    return 'null'

In [ ]:
rte_adhs['nft_avg_on_time'] = rte_adhs.apply(lambda row: categorise(row), axis=1)
display(rte_adhs)

#### all late first trips

In [ ]:
first_late_data = rte_adhs.loc[rte_adhs['on_time'] == 'late']
first_late_data.drop(first_late_data.columns.difference(['nft_avg_on_time']), axis=1, inplace=True)
display(first_late_data)

In [ ]:
fld = pd.DataFrame(first_late_data['nft_avg_on_time'].value_counts().reset_index())
display(fld)

In [ ]:
plt.pie(data=fld, x='nft_avg_on_time', labels='index', autopct='%.0f%%')